# The SPDZ Protocol

**Module 12** | 12-mpc

*Beaver triples, online/offline, malicious security*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **the spdz protocol**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [Oblivious Transfer](12d-oblivious-transfer.ipynb).
- Concepts and notation introduced in the previous notebook.

## Beaver Triples

In [ ]:
# Pre-shared random triples (a, b, c) with c = a*b
# Allow multiplication on shared values
# TODO: demonstrate Beaver triple generation

## Online Phase: Multiplication

In [ ]:
# To multiply [x]*[y]:
# Open d = x-a and e = y-b
# [xy] = [c] + d*[y] + e*[x] + d*e
# TODO: implement online multiplication

## Malicious Security

In [ ]:
# SPDZ uses MACs to detect cheating parties
# Each share has an authentication tag
# TODO: demonstrate MAC-based verification

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **the spdz protocol**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*